# 相关设置

In [35]:
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

InteractiveShell.ast_node_interactivity = "all"  # 一个cell显示多输出

pd.set_option('display.max_rows', 100)  # 设置最大显示100行
pd.set_option('display.max_columns', 100)  # 设置最大显示100列

In [36]:
EDA_data = pd.read_csv('data/after_EDA/EDA_data.csv')
print(EDA_data.shape)

(26956, 12)


In [37]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(EDA_data, test_size=0.2, random_state=412)

print(train_set.shape)
print(test_set.shape)

(21564, 12)
(5392, 12)


In [38]:
X_train = train_set[train_set.columns.difference(['is_pass'])].values
y_train = train_set[['is_pass']].values
X_test = test_set[test_set.columns.difference(['is_pass'])].values
y_test = test_set[['is_pass']].values

# 评估方法

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

# 随机森林

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

rf_model.fit(X_train, y_train.ravel())
y_predict = rf_model.predict(X=X_test)

print("accuracy_score: ", accuracy_score(y_test, y_predict))
print('confusion_matrix: ', confusion_matrix(y_test, y_predict))
print('roc_auc_score: ', roc_auc_score(y_test, y_predict))
print('classification_report: ', classification_report(y_test, y_predict))

RandomForestClassifier(n_estimators=10, n_jobs=1)

accuracy_score:  0.699740356083086
confusion_matrix:  [[ 747  918]
 [ 701 3026]]
roc_auc_score:  0.6302808577292076
classification_report:                precision    recall  f1-score   support

           0       0.52      0.45      0.48      1665
           1       0.77      0.81      0.79      3727

    accuracy                           0.70      5392
   macro avg       0.64      0.63      0.63      5392
weighted avg       0.69      0.70      0.69      5392



# LightGBM

In [41]:
import lightgbm as lgb

trn_data = lgb.Dataset(X_train, y_train)
val_data = lgb.Dataset(X_test, y_test)

params = {  
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 7,  
    'metric': 'multi_error',  
    'num_leaves': 120,  
    'min_data_in_leaf': 100,  
    'learning_rate': 0.06,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1, 
}

lgb_model = lgb.train(params, 
                      trn_data, 
                      num_boost_round = 1000,
                      valid_sets = [trn_data,val_data], 
                      verbose_eval = 100, 
                      early_stopping_rounds = 100)

y_prob = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
y_predict = [list(x).index(max(x)) for x in y_prob]

print("accuracy_score: ", accuracy_score(y_test, y_predict))
print('confusion_matrix: ', confusion_matrix(y_test, y_predict))
print('roc_auc_score: ', roc_auc_score(y_test, y_predict))
print('classification_report: ', classification_report(y_test, y_predict))

S:\DataMining\venv\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.250927	valid_1's multi_error: 0.284866
[200]	training's multi_error: 0.242719	valid_1's multi_error: 0.285052
Early stopping, best iteration is:
[150]	training's multi_error: 0.246939	valid_1's multi_error: 0.283197
accuracy_score:  0.7168026706231454
confusion_matrix:  [[ 533 1132]
 [ 395 3332]]
roc_auc_score:  0.6070683777418417
classification_report:                precision    recall  f1-score   support

           0       0.57      0.32      0.41      1665
           1       0.75      0.89      0.81      3727

    accuracy                           0.72      5392
   macro avg       0.66      0.61      0.61      5392
weighted avg       0.69      0.72      0.69      5392



# A榜提交数据生成

In [52]:
test_data = pd.read_csv('data/test/test1.csv')
test_data.head()

,id_num,program_type,program_id,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,1442_124,Y,Y_4,121,124,online,easy,1442,M,High School Diploma,2,NaN,8,N,4.0,NaN
1,12373_38,T,T_2,117,38,online,easy,12373,F,High School Diploma,4,31.0,2,N,3.0,NaN
2,12035_21,T,T_3,134,21,offline,easy,12035,F,High School Diploma,4,NaN,2,N,1.0,NaN
3,11148_86,V,V_3,131,86,offline,vary hard,11148,M,Matriculation,1,NaN,2,N,1.0,NaN
4,1362_131,Y,Y_3,135,131,online,easy,1362,M,High School Diploma,2,45.0,2,N,4.0,NaN


# 去除无用列

In [53]:
test_data.drop(['program_type', 'id_num', 'program_id', 'test_id', 'trainee_id'], inplace=True, axis=1)

# 缺失值填充

In [54]:
test_data['age'].fillna(test_data['age'].mean(), inplace=True)

# 独热编码

In [55]:
test_data = test_data.join(pd.get_dummies(test_data[["gender"]]))

# 去除独热编码的列

In [56]:
test_data.drop(['gender'], inplace=True, axis=1)

# 特征编码

In [57]:
test_type_map = {'online': 1, 'offline': 2}
difficulty_level_map = {'easy': 1, 'intermediate': 2, 'hard': 3, 'vary hard': 4}
education_map = {'No Qualification': 1, 'High School Diploma': 2, 'Matriculation': 3, 'Bachelors': 4, 'Masters': 5}
is_handicapped_map = {'N': 1, 'Y': 2}

In [58]:
test_data['test_type'] = test_data['test_type'].map(test_type_map)
test_data['difficulty_level'] = test_data['difficulty_level'].map(difficulty_level_map)
test_data['education'] = test_data['education'].map(education_map)
test_data['is_handicapped'] = test_data['is_handicapped'].map(is_handicapped_map)

In [59]:
test_data.drop(['is_pass'], axis=1, inplace=True)

In [60]:
test_data

,program_duration,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,gender_F,gender_M
0,121,1,1,2,2,36.561642,8,1,4.0,0,1
1,117,1,1,2,4,31.000000,2,1,3.0,1,0
2,134,2,1,2,4,36.561642,2,1,1.0,1,0
3,131,2,4,3,1,36.561642,2,1,1.0,0,1
4,135,1,1,2,2,45.000000,2,1,4.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
11679,130,2,1,2,3,36.561642,1,1,2.0,1,0
11680,117,1,1,2,2,45.000000,2,1,2.0,1,0
11681,120,1,1,3,2,36.561642,3,1,4.0,0,1
11682,131,2,2,2,3,27.000000,2,1,2.0,0,1


In [61]:
X_submit = test_data.values

In [62]:
y_prob = lgb_model.predict(X_submit, num_iteration=lgb_model.best_iteration)
y_submit = [list(x).index(max(x)) for x in y_prob]

In [65]:
# print(y_submit)

In [66]:
id_num = pd.read_csv('data/test/test1.csv')['id_num']

In [67]:
submission = pd.DataFrame({
        "id_num": id_num,
        "is_pass": y_submit
    })
submission.to_csv('data/submission_Giyn.csv', index=False)